In [1]:
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
pd.set_option('display.max_columns', None)


# Reading in and combining all of the game log data into a single DF

In [38]:
def call_game_logs():
    #returns single df containing game logs for the last 4 seasons
    
    #reading in each game_log df
    gl_2020_2021 = pd.read_csv('data/game_logs_current.csv', skipinitialspace=True)
    
    gl_2019_2020 = pd.read_csv('data/game_logs_2019_2020.csv', skipinitialspace=True)
    
    gl_2018_2019 = pd.read_csv('data/game_logs_2018_2019.csv', skipinitialspace=True)
    
    gl_2017_2018 = pd.read_csv('data/game_logs_2017_2018.csv', skipinitialspace=True)
    
    
    
    #dropping the Unnamed: 0 random columns from each df
    gl_2020_2021.drop('Unnamed: 0', axis = 1, inplace= True)
    
    gl_2019_2020.drop('Unnamed: 0', axis = 1, inplace= True)
    
    gl_2018_2019.drop('Unnamed: 0', axis = 1, inplace= True)
    
    gl_2017_2018.drop('Unnamed: 0', axis = 1, inplace= True)
    #concatenate all 3 game log df together
    
    #remeber to add in gl_2020_2021
    #!!!gl_2020_2021
    
    gl_full = pd.concat([gl_2019_2020, gl_2018_2019, gl_2017_2018])
    
    #replace "Game Date" column with a new column called ['Date']
    gl_full['Date'] = gl_full.iloc[:,2:3]
    gl_full.drop(gl_full.iloc[:,2:3], inplace = True, axis =1)
    
    
    #converting 'Date column to datetime'
    gl_full['Date'] = pd.to_datetime(gl_full['Date'])
    
    return gl_full
    
    
full = call_game_logs()

In [39]:
full

,Team,Match Up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Date
0,DEN,DEN @ TOR,L,240,109,36,87,41.4,16,38,42.1,21,23,91.3,9,32,41,27,9,11,16,15,-8,2020-08-14
1,TOR,TOR vs. DEN,W,240,117,45,90,50.0,18,35,51.4,9,14,64.3,13,38,51,27,8,5,19,24,8,2020-08-14
2,MIA,MIA @ IND,L,240,92,31,87,35.6,10,38,26.3,20,27,74.1,14,34,48,14,6,5,19,15,-17,2020-08-14
3,IND,IND vs. MIA,W,240,109,43,89,48.3,15,38,39.5,8,11,72.7,12,39,51,32,11,3,20,27,17,2020-08-14
4,HOU,HOU vs. PHI,L,240,96,35,80,43.8,12,48,25.0,14,16,87.5,1,26,27,25,10,9,17,18,-38,2020-08-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,SAS,SAS vs. MIN,W,240,107,41,89,46.1,8,21,38.1,17,19,89.5,12,38,50,22,8,11,11,18,8,2017-10-18
2456,CLE,CLE vs. BOS,W,240,102,38,83,45.8,5,22,22.7,21,25,84.0,9,41,50,19,3,4,17,25,3,2017-10-17
2457,BOS,BOS @ CLE,L,240,99,36,88,40.9,8,32,25.0,19,25,76.0,9,37,46,24,11,4,12,24,-3,2017-10-17
2458,GSW,GSW vs. HOU,L,240,121,43,80,53.8,16,30,53.3,19,21,90.5,6,35,41,34,5,9,17,25,-1,2017-10-17


# This function convertsthe int64 object to a datetime object to match with 
# game logs lf

In [136]:
#converting int64 to correct datetime object

def add_year(df, year1, year2):
    
    for idx, string in enumerate(df['Date']):
        
        #checking to see if the date is more than 3 in length because that means it is before january 1
        if len(string) > 3:
            df['Date'][idx] = year1 + '-' + string[:2] +'-' + string[2:]
        else:
            
            #For some reason it turned january 10 into a 0 so i had to
            #add in a 1 before the 0 to make it a 10
            if string[-1] == '0':
                df['Date'][idx]= year2 + '-' + string[:2] + '-' + '1' + string[-1]
            
            #if the len of the string is 3 that means it is january first or after so
            #i added 2018 before the month and '-' in between the month and day.
            else:
                df['Date'][idx] = year2 + '-' +string[:1] + '-'  + string[2:]
            
        
            
    return df['Date']




# Reading in and combing all of the betting information for last 4 seasons

In [137]:
def read_in_bet():
    
    #returns a full betting df of the regular season games for the last four seasons
    
    # reading in and indexing 2017_18 index these df here[:2460] to exclude the playoff betting information
    bet_2017_18 = pd.read_csv('data/nba_odds_2017-18.csv')
    bet_2017_18 = bet_2017_18[:2460]
    #Indexing to exclude the last columns 
    #we do not want
    bet_2017_18 = bet_2017_18.iloc[:,:13]
    
    #calling add year function to change the datetime object to match the gamelogs for 2018-2019 & converting 
    #'Date' to a string object
    bet_2017_18['Date'] = bet_2017_18['Date'].astype(str)
    bet_2017_18['Date'] = add_year(bet_2017_18, '2017', '2018')
    
    
    
    # reading in and indexing 2018_19 index these df here[:2460] to exclude the playoff betting information
    bet_2018_19 = pd.read_csv('data/nba_odds_2018_19.csv')
    bet_2018_19 = bet_2018_19[:2460]
    
    #calling add year function to change the datetime object to match the gamelogs for 2018-2019 & converting 
    #'Date' to a string object
    bet_2018_19['Date'] = bet_2018_19['Date'].astype(str)
    bet_2018_19['Date'] = add_year(bet_2018_19, '2018', '2019')

    
    
    #reading in and indexing 2019_2020 to exclude playoffs **covid season
    #This include covid year so index [:2118] to only include the regular season
    bet_2019_2020 = pd.read_csv('data/nba_odds_2019_20.csv')
    bet_2019_2020 = bet_2019_2020[:2118]
    
    #calling add year function to change the datetime object to match the gamelogs for 2019-2020 & converting 
    #'Date' to a string object
    bet_2019_2020['Date'] = bet_2019_2020['Date'].astype(str)
    bet_2019_2020['Date'] = add_year(bet_2019_2020, '2019', '2020')
    
    #just need to include the betting informatin for the current season
    ##### Finish when ready to run the model
    
    #creating a single DF for all of the seasons of betting information Indexing to exclude the last columns 
    #we do not want
    bet_full = pd.concat([bet_2019_2020, bet_2018_19, bet_2017_18])
    
    
    
    
    return bet_full
full_odds = read_in_bet()

/Users/amossworkcomp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/amossworkcomp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/amossworkcomp/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipyke

In [138]:
full_odds[1750:1800]

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
1750,2020-2-8,527,V,Minnesota,35,30,31,29,125,235,234,280,115
1751,2020-2-8,528,H,Orlando,35,34,34,33,136,7.5,8,-360,3.5
1752,2020-2-8,529,V,Charlotte,25,27,21,26,99,214.5,214,800,105.5
1753,2020-2-8,530,H,Toronto,23,18,25,30,96,13.5,13.5,-1400,13
1754,2020-2-8,531,V,Brooklyn,36,26,36,20,118,3,2.5,-140,3
1755,2020-2-8,532,H,Atlanta,29,39,40,33,141,232,236.5,120,121.5
1756,2020-2-8,533,V,Cleveland,25,30,28,21,104,237,235,650,113.5
1757,2020-2-8,534,H,NewOrleans,43,27,25,21,116,11.5,12.5,-1000,2.5
1758,2020-2-8,535,V,Sacramento,21,29,31,23,104,229.5,227,115,110
1759,2020-2-8,536,H,Memphis,30,17,28,26,101,3.5,2.5,-135,1.5


# Function to replace all the team string values to their acronym to combine with game log DF

In [8]:

def replace_string():
    
    
    full_odds['Team'] = full_odds['Team'].str.replace('GoldenState','GSW')
    full_odds['Team'] = full_odds['Team'].str.replace('Brooklyn', 'BKN')
    full_odds['Team'] = full_odds['Team'].str.replace('LAClippers', 'LAC')
    full_odds['Team'] = full_odds['Team'].str.replace('Charlotte', 'CHA')
    full_odds['Team'] = full_odds['Team'].str.replace('LALakers', 'LAL')
    full_odds['Team'] = full_odds['Team'].str.replace('Cleveland', 'CLE')
    full_odds['Team'] = full_odds['Team'].str.replace('NewYork','NYK')
    full_odds['Team'] = full_odds['Team'].str.replace('Indiana', 'IND')
    full_odds['Team'] = full_odds['Team'].str.replace('Miami', 'MIA')
    full_odds['Team'] = full_odds['Team'].str.replace('Orlando', 'ORL')
    full_odds['Team'] = full_odds['Team'].str.replace('Washington', 'WAS')
    full_odds['Team'] = full_odds['Team'].str.replace('Philadelphia', 'PHI')
    full_odds['Team'] = full_odds['Team'].str.replace('Milwaukee', 'MIL')
    full_odds['Team'] = full_odds['Team'].str.replace('Boston', 'BOS')
    full_odds['Team'] = full_odds['Team'].str.replace('NewOrleans', 'NOP')
    full_odds['Team'] = full_odds['Team'].str.replace('Toronto', 'TOR')
    full_odds['Team'] = full_odds['Team'].str.replace('Atlanta', 'ATL')
    full_odds['Team'] = full_odds['Team'].str.replace('Chicago', 'CHI')
    full_odds['Team'] = full_odds['Team'].str.replace('SanAntonio', 'SAS')
    full_odds['Team'] = full_odds['Team'].str.replace('Memphis', 'MEM')
    full_odds['Team'] = full_odds['Team'].str.replace('Detroit', 'DET')
    full_odds['Team'] = full_odds['Team'].str.replace('Minnesota', 'MIN')
    full_odds['Team'] = full_odds['Team'].str.replace('Sacramento', 'SAC')
    full_odds['Team'] = full_odds['Team'].str.replace('Denver', 'DEN')
    full_odds['Team'] = full_odds['Team'].str.replace('Utah', 'UTA')
    full_odds['Team'] = full_odds['Team'].str.replace('Portland', 'POR')
    full_odds['Team'] = full_odds['Team'].str.replace('Dallas', 'DAL')
    full_odds['Team'] = full_odds['Team'].str.replace('Phoenix', 'PHX')
    full_odds['Team'] = full_odds['Team'].str.replace('OklahomaCity', 'OKC')
    full_odds['Team'] = full_odds['Team'].str.replace('Houston', 'HOU')
    
    #converting Date in full odds to string to convert to datetime later 
    
    full_odds['Date']   =  full_odds['Date'].astype(str) 
    
    return full_odds
    
full_odd_real = replace_string()

In [9]:
full_odd_real

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
0,2019-10-22,501,V,NOP,30,31,25,31,122,231.5,229.5,230,113
1,2019-10-22,502,H,TOR,27,29,32,29,130,6.5,6.5,-280,6
2,2019-10-22,503,V,LAL,25,29,31,17,102,227,3.5,-180,5
3,2019-10-22,504,H,LAC,22,40,23,27,112,1.5,224,150,110.5
4,2019-10-23,505,V,DET,27,27,29,36,119,210,211,240,104
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,2018-4-1,720,H,POR,28,27,25,22,102,4,2,-135,105
2456,2018-4-1,721,V,HOU,22,22,19,20,83,6.5,201,115,97.5
2457,2018-4-1,722,H,SAC,28,24,27,17,96,204,2,-135,1
2458,2018-4-1,723,V,LAL,39,25,28,23,115,217,214.5,125,105


In [10]:
# # a list of all 30 teams in the nba to change into the acronym to combine dataframes later

# 'GoldenState' = 'GSW'
# 'Brooklyn' = 'BKN'
# 'LAClippers' = 'LAC'
# 'Charlotte' = 'CHA'
# 'LALakers' = 'LAL'
# 'Cleveland' = 'CLE'
# 'NewYork' = 'NYK'
# 'Indiana' = 'IND'
# 'Miami'  = 'MIA'
# 'Orlando' = 'ORL'
# 'Washington' = 'WAS'
# 'Philadelphia' = 'PHI'
# 'Milwaukee' = 'MIL'
# 'Boston' = 'BOS'
# 'NewOrleans' = 'NOP'
# 'Toronto' = 'TOR'
# 'Atlanta' = 'ATL'
# 'Chicago' = 'CHI'
# 'SanAntonio' = 'SAS'
# 'Memphis' = 'MEM'
# 'Detroit' = 'DET'
# 'Minnesota' = 'MIN'
# 'Sacramento' = 'SAC'
# 'Denver' = 'DEN'
# 'Utah' = 'UTA'
# 'Portland' = 'POR'
# 'Dallas' = 'DAL'
# 'Phoenix' = 'PHX'
# 'OklahomaCity' = 'OKC'
# 'Houston' = 'HOU'



In [11]:
# https://www.sportsbookreviewsonline.com/scoresoddsarchives/nba/nbaoddsarchives.htm

# Merging every other row in game log DF so each row represents single game

In [149]:
def combine_game_logs():
    
    #indexing every row and using concat to add to the row above
    merge_logs = pd.concat((full.iloc[::2].reset_index(drop=True), 
                    full.iloc[1::2].reset_index(drop=True)),
                   axis=1)
    
    
    #creating two different df to rename the second half of the columns
    old = merge_logs.iloc[:, :24]
    new = merge_logs.iloc[:, 24:]
    #adding suffix to column names in 2nd split DF
    new.columns = [str(col) + '_2' for col in new.columns]
    
    
    #re-merging the two DFs after adding a suffix to the second's column names
    merge_logs_final = pd.concat((old.reset_index(drop=True), 
                    new.reset_index(drop=True)),
                   axis=1)
    
    
    #converting the date column to a string value to merge with bet DF later
    merge_logs_final['Date'] = merge_logs_final['Date'].astype(str)
    
    
    #renamong [matchup column because it I couldnt access it previously]
    merge_logs_final = merge_logs_final.rename(columns={merge_logs_final.columns[1]: 'Match_up'})
    
    
    merge_logs_final['Total_pts'] = merge_logs_final['PTS'] + merge_logs_final['PTS_2']
    
    
    #creating a new column which is our y value that is total points for the game.
    return merge_logs_final

final_game_log =  combine_game_logs()

In [164]:
final_game_log

,Team,Match_up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Date,Team_2,Match Up_2,W/L_2,MIN_2,PTS_2,FGM_2,FGA_2,FG%_2,3PM_2,3PA_2,3P%_2,FTM_2,FTA_2,FT%_2,OREB_2,DREB_2,REB_2,AST_2,STL_2,BLK_2,TOV_2,PF_2,+/-_2,Date_2,Total_pts
0,DEN,DEN @ TOR,L,240,109,36,87,41.4,16,38,42.1,21,23,91.3,9,32,41,27,9,11,16,15,-8,2020-08-14,TOR,TOR vs. DEN,W,240,117,45,90,50.0,18,35,51.4,9,14,64.3,13,38,51,27,8,5,19,24,8,2020-08-14,226
1,MIA,MIA @ IND,L,240,92,31,87,35.6,10,38,26.3,20,27,74.1,14,34,48,14,6,5,19,15,-17,2020-08-14,IND,IND vs. MIA,W,240,109,43,89,48.3,15,38,39.5,8,11,72.7,12,39,51,32,11,3,20,27,17,2020-08-14,201
2,HOU,HOU vs. PHI,L,240,96,35,80,43.8,12,48,25.0,14,16,87.5,1,26,27,25,10,9,17,18,-38,2020-08-14,PHI,PHI @ HOU,W,240,134,49,87,56.3,18,38,47.4,18,22,81.8,8,43,51,31,8,4,16,19,38,2020-08-14,230
3,LAC,LAC vs. OKC,W,265,107,34,85,40.0,12,37,32.4,27,39,69.2,12,49,61,23,10,2,23,22,4,2020-08-14,OKC,OKC @ LAC,L,265,103,38,106,35.8,14,44,31.8,13,22,59.1,11,37,48,18,5,6,16,29,-4,2020-08-14,210
4,MIL,MIL @ MEM,L,240,106,39,98,39.8,13,41,31.7,15,23,65.2,9,36,45,27,8,3,15,22,-13,2020-08-13,MEM,MEM vs. MIL,W,240,119,45,90,50.0,13,33,39.4,16,23,69.6,9,46,55,36,4,3,18,18,13,2020-08-13,225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3514,BOS,BOS vs. MIL,L,240,100,39,91,42.9,11,28,39.3,11,21,52.4,11,32,43,23,12,2,15,27,-8,2017-10-18,MIL,MIL @ BOS,W,240,108,38,76,50.0,7,21,33.3,25,30,83.3,6,39,45,19,6,4,15,22,8,2017-10-18,208
3515,POR,POR @ PHX,W,240,124,44,90,48.9,14,24,58.3,22,28,78.6,15,42,57,22,8,7,18,20,48,2017-10-18,PHX,PHX vs. POR,L,240,76,28,89,31.5,7,27,25.9,13,18,72.2,10,23,33,10,9,8,16,22,-48,2017-10-18,200
3516,MIN,MIN @ SAS,L,240,99,37,85,43.5,8,19,42.1,17,23,73.9,11,31,42,23,7,4,13,16,-8,2017-10-18,SAS,SAS vs. MIN,W,240,107,41,89,46.1,8,21,38.1,17,19,89.5,12,38,50,22,8,11,11,18,8,2017-10-18,206
3517,CLE,CLE vs. BOS,W,240,102,38,83,45.8,5,22,22.7,21,25,84.0,9,41,50,19,3,4,17,25,3,2017-10-17,BOS,BOS @ CLE,L,240,99,36,88,40.9,8,32,25.0,19,25,76.0,9,37,46,24,11,4,12,24,-3,2017-10-17,201


,Team,Match_up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Date,Team_2,Match Up_2,W/L_2,MIN_2,PTS_2,FGM_2,FGA_2,FG%_2,3PM_2,3PA_2,3P%_2,FTM_2,FTA_2,FT%_2,OREB_2,DREB_2,REB_2,AST_2,STL_2,BLK_2,TOV_2,PF_2,+/-_2,Date_2
0,DEN,DEN @ TOR,L,240,109,36,87,41.4,16,38,42.1,21,23,91.3,9,32,41,27,9,11,16,15,-8,2020-08-14,TOR,TOR vs. DEN,W,240,117,45,90,50.0,18,35,51.4,9,14,64.3,13,38,51,27,8,5,19,24,8,2020-08-14
1,MIA,MIA @ IND,L,240,92,31,87,35.6,10,38,26.3,20,27,74.1,14,34,48,14,6,5,19,15,-17,2020-08-14,IND,IND vs. MIA,W,240,109,43,89,48.3,15,38,39.5,8,11,72.7,12,39,51,32,11,3,20,27,17,2020-08-14
2,HOU,HOU vs. PHI,L,240,96,35,80,43.8,12,48,25.0,14,16,87.5,1,26,27,25,10,9,17,18,-38,2020-08-14,PHI,PHI @ HOU,W,240,134,49,87,56.3,18,38,47.4,18,22,81.8,8,43,51,31,8,4,16,19,38,2020-08-14
3,LAC,LAC vs. OKC,W,265,107,34,85,40.0,12,37,32.4,27,39,69.2,12,49,61,23,10,2,23,22,4,2020-08-14,OKC,OKC @ LAC,L,265,103,38,106,35.8,14,44,31.8,13,22,59.1,11,37,48,18,5,6,16,29,-4,2020-08-14
4,MIL,MIL @ MEM,L,240,106,39,98,39.8,13,41,31.7,15,23,65.2,9,36,45,27,8,3,15,22,-13,2020-08-13,MEM,MEM vs. MIL,W,240,119,45,90,50.0,13,33,39.4,16,23,69.6,9,46,55,36,4,3,18,18,13,2020-08-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3514,BOS,BOS vs. MIL,L,240,100,39,91,42.9,11,28,39.3,11,21,52.4,11,32,43,23,12,2,15,27,-8,2017-10-18,MIL,MIL @ BOS,W,240,108,38,76,50.0,7,21,33.3,25,30,83.3,6,39,45,19,6,4,15,22,8,2017-10-18
3515,POR,POR @ PHX,W,240,124,44,90,48.9,14,24,58.3,22,28,78.6,15,42,57,22,8,7,18,20,48,2017-10-18,PHX,PHX vs. POR,L,240,76,28,89,31.5,7,27,25.9,13,18,72.2,10,23,33,10,9,8,16,22,-48,2017-10-18
3516,MIN,MIN @ SAS,L,240,99,37,85,43.5,8,19,42.1,17,23,73.9,11,31,42,23,7,4,13,16,-8,2017-10-18,SAS,SAS vs. MIN,W,240,107,41,89,46.1,8,21,38.1,17,19,89.5,12,38,50,22,8,11,11,18,8,2017-10-18
3517,CLE,CLE vs. BOS,W,240,102,38,83,45.8,5,22,22.7,21,25,84.0,9,41,50,19,3,4,17,25,3,2017-10-17,BOS,BOS @ CLE,L,240,99,36,88,40.9,8,32,25.0,19,25,76.0,9,37,46,24,11,4,12,24,-3,2017-10-17


In [15]:
def combine_game_odd():
    
    #indexing every row and using concat to add to the row above
    merge_odds = pd.concat((full_odd_real.iloc[::2].reset_index(drop=True), 
                    full_odd_real.iloc[1::2].reset_index(drop=True)),
                   axis=1)
    
    
    #creating two different df to rename the second half of the columns
    old = merge_odds.iloc[:, :13]
    new = merge_odds.iloc[:, 13:]
    #adding suffix to column names in 2nd split DF
    new.columns = [str(col) + '_2' for col in new.columns]
    
    
    #re-merging the two DFs after adding a suffix to the second's column names
    merge_odds_final = pd.concat((old.reset_index(drop=True), 
                    new.reset_index(drop=True)),
                   axis=1)
    
    
    
    return merge_odds_final



final_odds = combine_game_odd()

In [139]:
final_odds['Total_pts'] = final_odds['Final'] + final_odds['Final_2']


In [170]:
#renaming columns to identify home team or away team
final_odds.rename(columns={'Team':'visitor'}, inplace=True)
final_odds.rename(columns={'Team_2':'home'}, inplace=True)

In [29]:
merged_df = final_game_logs.merge(final_odds, how = 'left', on = ['Date', 'Team'])

In [260]:
#creating new columns so that we can join bet DF with game_logs df 

final_game_log['home'] = [final_game_log['Team'][i] if '@' not in v else final_game_log['Team_2'][i] for i, v in enumerate(final_game_log['Match_up'])]
final_game_log['visitor'] = [final_game_log['Team'][i] if '@' in v else final_game_log['Team_2'][i] for i, v in enumerate(final_game_log['Match_up'])]

In [261]:
final_game_log

,Team,Match_up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Date,Team_2,Match Up_2,W/L_2,MIN_2,PTS_2,FGM_2,FGA_2,FG%_2,3PM_2,3PA_2,3P%_2,FTM_2,FTA_2,FT%_2,OREB_2,DREB_2,REB_2,AST_2,STL_2,BLK_2,TOV_2,PF_2,+/-_2,Date_2,Total_pts,visitor,home
0,DEN,DEN @ TOR,L,240,109,36,87,41.4,16,38,42.1,21,23,91.3,9,32,41,27,9,11,16,15,-8,2020-08-14,TOR,TOR vs. DEN,W,240,117,45,90,50.0,18,35,51.4,9,14,64.3,13,38,51,27,8,5,19,24,8,2020-08-14,226,DEN,TOR
1,MIA,MIA @ IND,L,240,92,31,87,35.6,10,38,26.3,20,27,74.1,14,34,48,14,6,5,19,15,-17,2020-08-14,IND,IND vs. MIA,W,240,109,43,89,48.3,15,38,39.5,8,11,72.7,12,39,51,32,11,3,20,27,17,2020-08-14,201,MIA,IND
2,HOU,HOU vs. PHI,L,240,96,35,80,43.8,12,48,25.0,14,16,87.5,1,26,27,25,10,9,17,18,-38,2020-08-14,PHI,PHI @ HOU,W,240,134,49,87,56.3,18,38,47.4,18,22,81.8,8,43,51,31,8,4,16,19,38,2020-08-14,230,PHI,HOU
3,LAC,LAC vs. OKC,W,265,107,34,85,40.0,12,37,32.4,27,39,69.2,12,49,61,23,10,2,23,22,4,2020-08-14,OKC,OKC @ LAC,L,265,103,38,106,35.8,14,44,31.8,13,22,59.1,11,37,48,18,5,6,16,29,-4,2020-08-14,210,OKC,LAC
4,MIL,MIL @ MEM,L,240,106,39,98,39.8,13,41,31.7,15,23,65.2,9,36,45,27,8,3,15,22,-13,2020-08-13,MEM,MEM vs. MIL,W,240,119,45,90,50.0,13,33,39.4,16,23,69.6,9,46,55,36,4,3,18,18,13,2020-08-13,225,MIL,MEM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3514,BOS,BOS vs. MIL,L,240,100,39,91,42.9,11,28,39.3,11,21,52.4,11,32,43,23,12,2,15,27,-8,2017-10-18,MIL,MIL @ BOS,W,240,108,38,76,50.0,7,21,33.3,25,30,83.3,6,39,45,19,6,4,15,22,8,2017-10-18,208,MIL,BOS
3515,POR,POR @ PHX,W,240,124,44,90,48.9,14,24,58.3,22,28,78.6,15,42,57,22,8,7,18,20,48,2017-10-18,PHX,PHX vs. POR,L,240,76,28,89,31.5,7,27,25.9,13,18,72.2,10,23,33,10,9,8,16,22,-48,2017-10-18,200,POR,PHX
3516,MIN,MIN @ SAS,L,240,99,37,85,43.5,8,19,42.1,17,23,73.9,11,31,42,23,7,4,13,16,-8,2017-10-18,SAS,SAS vs. MIN,W,240,107,41,89,46.1,8,21,38.1,17,19,89.5,12,38,50,22,8,11,11,18,8,2017-10-18,206,MIN,SAS
3517,CLE,CLE vs. BOS,W,240,102,38,83,45.8,5,22,22.7,21,25,84.0,9,41,50,19,3,4,17,25,3,2017-10-17,BOS,BOS @ CLE,L,240,99,36,88,40.9,8,32,25.0,19,25,76.0,9,37,46,24,11,4,12,24,-3,2017-10-17,201,BOS,CLE


In [263]:
merged_df_try1 = final_game_log.merge(final_odds, how = 'left', on = ['visitor', 'home', 'Total_pts'])

In [267]:
merged_df_try1

,Team,Match_up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Date_x,Team_2,Match Up_2,W/L_2,MIN_2,PTS_2,FGM_2,FGA_2,FG%_2,3PM_2,3PA_2,3P%_2,FTM_2,FTA_2,FT%_2,OREB_2,DREB_2,REB_2,AST_2,STL_2,BLK_2,TOV_2,PF_2,+/-_2,Date_2_x,Total_pts,visitor,home,Date_y,Rot,VH,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,Date_2_y,Rot_2,VH_2,1st_2,2nd_2,3rd_2,4th_2,Final_2,Open_2,Close_2,ML_2,2H_2
0,DEN,DEN @ TOR,L,240,109,36,87,41.4,16,38,42.1,21,23,91.3,9,32,41,27,9,11,16,15,-8,2020-08-14,TOR,TOR vs. DEN,W,240,117,45,90,50.0,18,35,51.4,9,14,64.3,13,38,51,27,8,5,19,24,8,2020-08-14,226,DEN,TOR,2020-8-4,717.0,N,23.0,27.0,26.0,33.0,109.0,2.5,2.5,-145.0,3,2020-8-4,718.0,N,31.0,27.0,27.0,32.0,117.0,224,223.5,125.0,110.5
1,MIA,MIA @ IND,L,240,92,31,87,35.6,10,38,26.3,20,27,74.1,14,34,48,14,6,5,19,15,-17,2020-08-14,IND,IND vs. MIA,W,240,109,43,89,48.3,15,38,39.5,8,11,72.7,12,39,51,32,11,3,20,27,17,2020-08-14,201,MIA,IND,2020-8-4,721.0,N,13.0,31.0,26.0,22.0,92.0,1,2,-130.0,3,2020-8-4,722.0,N,28.0,30.0,26.0,25.0,109.0,216,221.5,110.0,109.5
2,HOU,HOU vs. PHI,L,240,96,35,80,43.8,12,48,25.0,14,16,87.5,1,26,27,25,10,9,17,18,-38,2020-08-14,PHI,PHI @ HOU,W,240,134,49,87,56.3,18,38,47.4,18,22,81.8,8,43,51,31,8,4,16,19,38,2020-08-14,230,PHI,HOU,2020-8-4,715.0,N,39.0,32.0,32.0,31.0,134.0,228.5,230.5,120.0,113,2020-8-4,716.0,N,26.0,21.0,30.0,19.0,96.0,4.5,2.5,-140.0,7.5
3,LAC,LAC vs. OKC,W,265,107,34,85,40.0,12,37,32.4,27,39,69.2,12,49,61,23,10,2,23,22,4,2020-08-14,OKC,OKC @ LAC,L,265,103,38,106,35.8,14,44,31.8,13,22,59.1,11,37,48,18,5,6,16,29,-4,2020-08-14,210,OKC,LAC,2020-8-4,719.0,N,22.0,25.0,16.0,24.0,103.0,222,224.5,135.0,107.5,2020-8-4,720.0,N,21.0,16.0,33.0,17.0,107.0,6,3,-155.0,3.5
4,MIL,MIL @ MEM,L,240,106,39,98,39.8,13,41,31.7,15,23,65.2,9,36,45,27,8,3,15,22,-13,2020-08-13,MEM,MEM vs. MIL,W,240,119,45,90,50.0,13,33,39.4,16,23,69.6,9,46,55,36,4,3,18,18,13,2020-08-13,225,MIL,MEM,2020-8-3,703.0,N,21.0,29.0,24.0,32.0,106.0,223.5,231,145.0,0.5,2020-8-3,704.0,N,22.0,38.0,33.0,26.0,119.0,1,4,-165.0,117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3680,BOS,BOS vs. MIL,L,240,100,39,91,42.9,11,28,39.3,11,21,52.4,11,32,43,23,12,2,15,27,-8,2017-10-18,MIL,MIL @ BOS,W,240,108,38,76,50.0,7,21,33.3,25,30,83.3,6,39,45,19,6,4,15,22,8,2017-10-18,208,MIL,BOS,2017-10-18,709.0,V,24.0,34.0,18.0,32.0,108.0,208.5,203,105.0,101,2017-10-18,710.0,H,26.0,27.0,27.0,20.0,100.0,6,1.5,-125.0,2
3681,POR,POR @ PHX,W,240,124,44,90,48.9,14,24,58.3,22,28,78.6,15,42,57,22,8,7,18,20,48,2017-10-18,PHX,PHX vs. POR,L,240,76,28,89,31.5,7,27,25.9,13,18,72.2,10,23,33,10,9,8,16,22,-48,2017-10-18,200,POR,PHX,2017-10-18,721.0,V,29.0,31.0,38.0,26.0,124.0,1,2.5,-140.0,113,2017-10-18,722.0,H,21.0,14.0,20.0,21.0,76.0,220,219.5,120.0,5
3682,MIN,MIN @ SAS,L,240,99,37,85,43.5,8,19,42.1,17,23,73.9,11,31,42,23,7,4,13,16,-8,2017-10-18,SAS,SAS vs. MIN,W,240,107,41,89,46.1,8,21,38.1,17,19,89.5,12,38,50,22,8,11,11,18,8,2017-10-18,206,MIN,SAS,2017-10-18,717.0,V,25.0,24.0,28.0,22.0,99.0,207,203.5,105.0,1,2017-10-18,718.0,H,32.0,21.0,34.0,20.0,107.0,3,1.5,-125.0,100
3683,CLE,CLE vs. BOS,W,240,102,38,83,45.8,5,22,22.7,21,25,84.0,9,41,50,19,3,4,17,25,3,2017-10-17,BOS,BOS @ CLE,L,240,99,36,88,40.9,8,32,25.0,19,25,76.0,9,37,46,24,11,4,12,24,-3,2017-10-17,201,BOS,CLE,2017-10-17,501.0,V,19.0,19.0,33.0,28.0,99.0,214.5,216,166.0,1.5,2017-10-17,502.0,H,29.0,25.0,18.0,30.0,102.0,4,4.5,-195.0,110


In [165]:
final_game_
final_game_log

,Team,Match_up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,Date,Team_2,Match Up_2,W/L_2,MIN_2,PTS_2,FGM_2,FGA_2,FG%_2,3PM_2,3PA_2,3P%_2,FTM_2,FTA_2,FT%_2,OREB_2,DREB_2,REB_2,AST_2,STL_2,BLK_2,TOV_2,PF_2,+/-_2,Date_2,Total_pts
0,DEN,DEN @ TOR,L,240,109,36,87,41.4,16,38,42.1,21,23,91.3,9,32,41,27,9,11,16,15,-8,2020-08-14,TOR,TOR vs. DEN,W,240,117,45,90,50.0,18,35,51.4,9,14,64.3,13,38,51,27,8,5,19,24,8,2020-08-14,226
1,MIA,MIA @ IND,L,240,92,31,87,35.6,10,38,26.3,20,27,74.1,14,34,48,14,6,5,19,15,-17,2020-08-14,IND,IND vs. MIA,W,240,109,43,89,48.3,15,38,39.5,8,11,72.7,12,39,51,32,11,3,20,27,17,2020-08-14,201
2,HOU,HOU vs. PHI,L,240,96,35,80,43.8,12,48,25.0,14,16,87.5,1,26,27,25,10,9,17,18,-38,2020-08-14,PHI,PHI @ HOU,W,240,134,49,87,56.3,18,38,47.4,18,22,81.8,8,43,51,31,8,4,16,19,38,2020-08-14,230
3,LAC,LAC vs. OKC,W,265,107,34,85,40.0,12,37,32.4,27,39,69.2,12,49,61,23,10,2,23,22,4,2020-08-14,OKC,OKC @ LAC,L,265,103,38,106,35.8,14,44,31.8,13,22,59.1,11,37,48,18,5,6,16,29,-4,2020-08-14,210
4,MIL,MIL @ MEM,L,240,106,39,98,39.8,13,41,31.7,15,23,65.2,9,36,45,27,8,3,15,22,-13,2020-08-13,MEM,MEM vs. MIL,W,240,119,45,90,50.0,13,33,39.4,16,23,69.6,9,46,55,36,4,3,18,18,13,2020-08-13,225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3514,BOS,BOS vs. MIL,L,240,100,39,91,42.9,11,28,39.3,11,21,52.4,11,32,43,23,12,2,15,27,-8,2017-10-18,MIL,MIL @ BOS,W,240,108,38,76,50.0,7,21,33.3,25,30,83.3,6,39,45,19,6,4,15,22,8,2017-10-18,208
3515,POR,POR @ PHX,W,240,124,44,90,48.9,14,24,58.3,22,28,78.6,15,42,57,22,8,7,18,20,48,2017-10-18,PHX,PHX vs. POR,L,240,76,28,89,31.5,7,27,25.9,13,18,72.2,10,23,33,10,9,8,16,22,-48,2017-10-18,200
3516,MIN,MIN @ SAS,L,240,99,37,85,43.5,8,19,42.1,17,23,73.9,11,31,42,23,7,4,13,16,-8,2017-10-18,SAS,SAS vs. MIN,W,240,107,41,89,46.1,8,21,38.1,17,19,89.5,12,38,50,22,8,11,11,18,8,2017-10-18,206
3517,CLE,CLE vs. BOS,W,240,102,38,83,45.8,5,22,22.7,21,25,84.0,9,41,50,19,3,4,17,25,3,2017-10-17,BOS,BOS @ CLE,L,240,99,36,88,40.9,8,32,25.0,19,25,76.0,9,37,46,24,11,4,12,24,-3,2017-10-17,201


In [ ]:
def get_features():
    
    grouped = final_game_log.groupby(['Team']).mean()
    
    

In [166]:
# grouped = final_game_log.groupby(['Team']).mean()
# grouped

In [222]:
final_odds

,Date,Rot,VH,visitor,1st,2nd,3rd,4th,Final,Open,Close,ML,2H,Date_2,Rot_2,VH_2,home,1st_2,2nd_2,3rd_2,4th_2,Final_2,Open_2,Close_2,ML_2,2H_2,Total_pts
0,2019-10-22,501,V,NOP,30,31,25,31,122,231.5,229.5,230,113,2019-10-22,502,H,TOR,27,29,32,29,130,6.5,6.5,-280,6,252
1,2019-10-22,503,V,LAL,25,29,31,17,102,227,3.5,-180,5,2019-10-22,504,H,LAC,22,40,23,27,112,1.5,224,150,110.5,214
2,2019-10-23,505,V,DET,27,27,29,36,119,210,211,240,104,2019-10-23,506,H,IND,24,31,31,24,110,5,7,-300,4.5,229
3,2019-10-23,507,V,CLE,24,17,24,20,85,212.5,211.5,400,108,2019-10-23,508,H,ORL,28,27,16,23,94,7.5,9.5,-550,1,179
4,2019-10-23,509,V,CHI,28,27,40,30,125,1.5,3.5,-170,4,2019-10-23,510,H,CHA,37,26,33,30,126,218.5,216.5,145,112,251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3514,2018-4-1,715,V,BKN,22,21,27,27,97,207,4.5,-195,6.5,2018-4-1,716,H,BOS,27,26,30,27,110,5.5,205.5,165,106,207
3515,2018-4-1,717,V,DET,35,31,28,25,119,3,5,-200,103,2018-4-1,718,H,CHI,21,29,24,13,87,212.5,212.5,170,2,206
3516,2018-4-1,719,V,UTA,21,20,18,34,93,201.5,201.5,115,3.5,2018-4-1,720,H,POR,28,27,25,22,102,4,2,-135,105,195
3517,2018-4-1,721,V,HOU,22,22,19,20,83,6.5,201,115,97.5,2018-4-1,722,H,SAC,28,24,27,17,96,204,2,-135,1,179
